In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,51013
2,Huelva,Confirmados PDIA 14 días,3020
3,Huelva,Tasa PDIA 14 días,"588,4989379737709"
4,Huelva,Confirmados PDIA 7 días,1729
5,Huelva,Tasa PDIA 7 dias,"336,92538534988404"
6,Huelva,Total Confirmados,51229
7,Huelva,Curados,46900
8,Huelva,Fallecidos,425


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  51013.0


/tmp/ipykernel_39080/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  14358.0


/tmp/ipykernel_39080/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_39080/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_39080/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_39080/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 241 personas en los últimos 7 días 

Un positivo PCR cada 141 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,51013.0,1729.0,3020.0,513170.0,336.925385,588.498938,429.0
Huelva-Costa,30101.0,1034.0,1749.0,289548.0,357.108321,604.044925,295.0
Huelva (capital),14358.0,598.0,1020.0,143837.0,415.748382,709.136036,158.0
Sierra de Huelva-Andévalo Central,4751.0,296.0,520.0,67391.0,439.227790,771.616388,69.0
Condado-Campiña,15728.0,395.0,746.0,156231.0,252.830744,477.498064,64.0
Ayamonte,2259.0,97.0,129.0,21104.0,459.628506,611.258529,40.0
Aljaraque,1877.0,96.0,161.0,21474.0,447.052249,749.743876,32.0
Aracena,884.0,98.0,202.0,8255.0,1187.159297,2447.001817,22.0
Moguer,2248.0,56.0,112.0,21867.0,256.093657,512.187314,17.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Puerto Moral,19.0,5.0,14.0,278.0,1798.561151,5035.971223,1.0
Cumbres Mayores,188.0,18.0,48.0,1749.0,1029.159520,2744.425386,4.0
Aracena,884.0,98.0,202.0,8255.0,1187.159297,2447.001817,22.0
Fuenteheridos,29.0,9.0,14.0,709.0,1269.393512,1974.612130,6.0
Marines (Los),22.0,5.0,7.0,399.0,1253.132832,1754.385965,0.0
Villanueva de las Cruces,27.0,2.0,6.0,387.0,516.795866,1550.387597,1.0
Corteconcepción,56.0,5.0,7.0,536.0,932.835821,1305.970149,0.0
Almonaster la Real,113.0,15.0,20.0,1788.0,838.926174,1118.568233,5.0
Manzanilla,171.0,18.0,19.0,2118.0,849.858357,897.072710,2.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Villarrasa,249.0,4.0,13.0,2211.0,180.913614,587.969245,0.0,0.307692
Villanueva de las Cruces,27.0,2.0,6.0,387.0,516.795866,1550.387597,1.0,0.333333
Puerto Moral,19.0,5.0,14.0,278.0,1798.561151,5035.971223,1.0,0.357143
Cumbres Mayores,188.0,18.0,48.0,1749.0,1029.159520,2744.425386,4.0,0.375000
Palma del Condado (La),1527.0,14.0,36.0,10801.0,129.617628,333.302472,2.0,0.388889
Isla Cristina,3265.0,27.0,64.0,21393.0,126.209508,299.163278,10.0,0.421875
Almendro (El),64.0,3.0,7.0,840.0,357.142857,833.333333,2.0,0.428571
Almonte,2501.0,49.0,113.0,24507.0,199.942873,461.092749,7.0,0.433628
Paterna del Campo,292.0,8.0,18.0,3457.0,231.414521,520.682673,1.0,0.444444
